In [17]:
# import things
import os
import pandas as pd
import numpy as np

In [18]:
# read in csv
data = pd.read_csv('credit_record.csv')

In [19]:
data.head(100)

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
95,5001715,-35,X
96,5001715,-36,X
97,5001715,-37,X
98,5001715,-38,X


In [20]:
# data['STATUS'].unique()
# status = {'X' : 'No Loan',
#           'C' : 'Paid',
#          '0' : '1-29 days overdue',
#          '1' : '30-59 days overdue', 
#          '2' : '60-89 days overdue',
#          '3' : '90-119 days overdue',
#          '4' : '120-149 days overdue',
#          '5' : '150 days or more overdue'}

#give all of the status numerical values
data['STATUS'].unique()
status = {'X' : 0,
          'C' : 0,
         '0' : 1,
         '1' : 2, 
         '2' : 3,
         '3' : 4,
         '4' : 5,
         '5' : 6}

In [21]:
# replace them with numerical values in the DF 
data.replace({'STATUS': status}, inplace=True)

In [22]:
# data['STATUS'] = data['STATUS'].astype('category')
data.dtypes

ID                int64
MONTHS_BALANCE    int64
STATUS            int64
dtype: object

In [23]:
# make the months positive not negative
data['MONTHS_BALANCE'] = data['MONTHS_BALANCE']*-1

data

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,0
1,5001711,1,1
2,5001711,2,1
3,5001711,3,1
4,5001712,0,0
...,...,...,...
1048570,5150487,25,0
1048571,5150487,26,0
1048572,5150487,27,0
1048573,5150487,28,0


In [24]:
data['MONTHS_BALANCE'].unique()
# copy this df so i can use them for a pivot table
a = data.copy(deep=True)

In [25]:
# pivot table
newf = a.pivot(index='ID', columns = 'MONTHS_BALANCE')

# since not everyone paid in the same amount of months, fill nan values with 0 value
newf = newf.replace(np.nan,0)
newf.head()

# drop multilevel headers and flatten
newf.columns = newf.columns.droplevel(0)

# create new column and sum up all values within each row
newf['SUM'] = newf.sum(axis=1)
newf.head()
newf.columns = newf.columns.get_level_values(0)

# newf.columns = [ 'Month_'.join([str(c) for c in c_list]) for c_list in newf.columns.values ]
# newf.columns = newf.columns.droplevel(1)

# reset the index because this wll drive me nuts
newf = newf.reset_index()

# newf.rename(index={'MONTHS_BALANCE': 'Index'})

In [26]:
newf.head()

newf['SUM'] = newf['SUM'].astype(int)

In [27]:
newf['APPROVAL'] = ''

# for i in newf['SUM']:
#     if newf['SUM'] > 5:
#         newf['APPROVAL'] = 'NO'
#     else:
#         newf['APPROVAL'] = 'YES'

In [28]:
newf.head()
newf.dtypes

MONTHS_BALANCE
ID            int64
0           float64
1           float64
2           float64
3           float64
             ...   
58          float64
59          float64
60          float64
SUM           int64
APPROVAL     object
Length: 64, dtype: object

In [29]:
num = newf['SUM']

for i in num:
    if i >= 4:
        newf['APPROVAL'] = 'NO'
    else:
        newf['APPROVAL'] = 'YES'
        
newf.head()

MONTHS_BALANCE,ID,0,1,2,3,4,5,6,7,8,...,53,54,55,56,57,58,59,60,SUM,APPROVAL
0,5001711,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,YES
1,5001712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,YES
2,5001713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,YES
3,5001714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,YES
4,5001715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,YES


In [54]:
# newf.to_csv('credit_reportCLEAN.csv', header=None)

In [62]:
newf.to_csv('CRUpdate.csv')

[No Loan, 1-29 days overdue, Paid, 30-59 days overdue, 60-89 days overdue, 90-119 days overdue, 120-149 days overdue, 150 days or more overdue]
Categories (8, object): [No Loan, 1-29 days overdue, Paid, 30-59 days overdue, 60-89 days overdue, 90-119 days overdue, 120-149 days overdue, 150 days or more overdue]